In [1]:
import pandas as pd

In [2]:
state = pd.read_csv("datasets/electric_vehicle_sales_by_state.csv")
makers = pd.read_csv("datasets/electric_vehicle_sales_by_makers.csv")
dim = pd.read_csv("datasets/dim_date.csv")

In [4]:
state["date"] = pd.to_datetime(state["date"])
makers["date"] = pd.to_datetime(makers["date"])
dim["date"] = pd.to_datetime(dim["date"])

In [5]:
state = pd.merge(state,dim, on = "date")
makers = pd.merge(makers,dim, on = "date")

# 1

In [6]:
def top_bottom_maker(dt,fiscal_year):
    required_dt = dt[(dt["vehicle_category"] == "2-Wheelers") & (dt["fiscal_year"] == fiscal_year)][["maker","electric_vehicles_sold"]]
    final_dt = required_dt.groupby("maker").sum()
    top = final_dt.sort_values(by = "electric_vehicles_sold", ascending = False).head(3)
    bottom = final_dt.sort_values(by = "electric_vehicles_sold").head(3)
    return [top,bottom]

In [7]:
top_maker_2023,bottom_maker_2023 = top_bottom_maker(makers.copy(),2023)
top_maker_2024,bottom_maker_2024 = top_bottom_maker(makers.copy(),2024)

In [8]:
top_maker_2023

,electric_vehicles_sold
maker,
OLA ELECTRIC,152583
OKINAWA,96945
HERO ELECTRIC,88993


In [9]:
top_maker_2024

,electric_vehicles_sold
maker,
OLA ELECTRIC,322489
TVS,180743
ATHER,107552


In [10]:
bottom_maker_2023

,electric_vehicles_sold
maker,
JITENDRA,8563
BEING,11018
PURE EV,11556


In [11]:
bottom_maker_2024

,electric_vehicles_sold
maker,
BATTRE ELECTRIC,4841
REVOLT,7254
KINETIC GREEN,9585


# 2

In [13]:
dt = state.copy()
fiscal_year = 2024
required_dt = dt[dt["fiscal_year"] == fiscal_year][["state","electric_vehicles_sold","total_vehicles_sold"]]
final_dt = required_dt.groupby("state").sum()
final_dt["penetration_rate"] = (final_dt["electric_vehicles_sold"]/final_dt["total_vehicles_sold"])*100
top_5_states_with_the_highest_penetration_rate = final_dt.sort_values(by = "penetration_rate", ascending = False).head(5)

In [14]:
top_5_states_with_the_highest_penetration_rate

,electric_vehicles_sold,total_vehicles_sold,penetration_rate
state,,,
Goa,10799,78524,13.752483
Kerala,73938,638114,11.586958
Karnataka,160989,1581988,10.176373
Maharashtra,197169,2293994,8.595009
Delhi,46724,606348,7.705806


# 3

In [15]:
dt = state[(state["fiscal_year"] == 2022) | (state["fiscal_year"] == 2024)][["state","electric_vehicles_sold","total_vehicles_sold","fiscal_year"]]

In [16]:
temp_dt = dt.groupby(["state","fiscal_year"]).sum().unstack()

In [17]:
temp_dt["penetration_rate_2022"] =  (temp_dt["electric_vehicles_sold"][2022]/temp_dt["total_vehicles_sold"][2022])*100
temp_dt["penetration_rate_2024"] =  (temp_dt["electric_vehicles_sold"][2024]/temp_dt["total_vehicles_sold"][2024])*100

In [18]:
temp_dt["penetration_2022>2024"] = temp_dt["penetration_rate_2022"] > temp_dt["penetration_rate_2024"]

In [19]:
final_dt = temp_dt.reset_index()[["state","penetration_rate_2022","penetration_rate_2024","penetration_2022>2024"]].stack().reset_index()[["state","penetration_rate_2022","penetration_rate_2024","penetration_2022>2024"]].set_index("state")

In [20]:
final_dt

,penetration_rate_2022,penetration_rate_2024,penetration_2022>2024
state,,,
Andaman & Nicobar,NaN,0.303030,False
Andaman & Nicobar Island,0.427350,0.504356,False
Andhra Pradesh,1.802399,4.238662,False
Arunachal Pradesh,0.000000,0.111143,False
Assam,0.192384,0.638575,False
Bihar,0.540838,1.330358,False
Chandigarh,1.112194,6.372516,False
Chhattisgarh,1.161754,5.673189,False
DNH and DD,0.281962,1.207317,False


# 4

In [21]:
df_1 = makers[makers["vehicle_category"] == "4-Wheelers"][["maker","electric_vehicles_sold"]]

In [22]:
top_5 = list(df_1.groupby("maker").sum().sort_values(by = "electric_vehicles_sold", ascending = False).head(5).index)

In [23]:
df_2 = makers[["maker","electric_vehicles_sold","quarter","fiscal_year"]][makers["maker"].isin(top_5)]

In [24]:
final_dt = df_2.groupby(["maker","quarter","fiscal_year"]).sum().unstack().unstack()

In [25]:
final_dt

electric_vehicles_sold                                \
fiscal_year                           2022                    2023         
quarter                                 Q1    Q2    Q3    Q4    Q1    Q2   
maker                                                                      
BYD India                                0     0     1    32    81   113   
Hyundai Motor                           25    34    25    26    75   155   
MG Motor                               285   798   411   153   531   635   
Mahindra & Mahindra                    355   651  1383  1653  2020  3164   
Tata Motors                           1031  2052  3791  5834  5675  6192   

                                                             
fiscal_year                       2024                       
quarter                Q3    Q4     Q1     Q2     Q3     Q4  
maker                                                        
BYD India             103   623    406    310    350    400  
Hyundai Motor         191   155    292    390    370    338  
MG Motor             1165   946   1493   2524   2190   2622  
Mahindra & Mahindra  3378  5243  10911   5855   4264   2316  
Tata Motors          6651  9528   7247  10337  13236  17361

# 5

In [26]:
raw_data = state[((state["state"] == "Delhi")|(state["state"] == "Karnataka"))&((state["fiscal_year"] == 2024))].drop(columns = ["fiscal_year","date"])

In [27]:
raw_data.sample(5)

,state,vehicle_category,electric_vehicles_sold,total_vehicles_sold,quarter
1785,Delhi,4-Wheelers,935,15723,Q1
2130,Karnataka,4-Wheelers,822,26232,Q3
2085,Delhi,2-Wheelers,2611,35861,Q3
2159,Karnataka,2-Wheelers,14300,136535,Q3
2058,Delhi,4-Wheelers,519,16459,Q3


In [28]:
ev_sales = raw_data[["state","electric_vehicles_sold"]].groupby("state").sum()

In [29]:
ev_sales

,electric_vehicles_sold
state,
Delhi,46724
Karnataka,160989


In [30]:
ev_sales_by_category = raw_data.drop(columns = ["total_vehicles_sold","quarter"]).groupby(["state","vehicle_category"]).sum().sort_values(by = "vehicle_category")

In [31]:
ev_sales_by_category

,,electric_vehicles_sold
state,vehicle_category,
Delhi,2-Wheelers,38094
Karnataka,2-Wheelers,148111
Delhi,4-Wheelers,8630
Karnataka,4-Wheelers,12878


In [32]:
quarterly_ev_sales = raw_data.drop(columns = "total_vehicles_sold").groupby(["state","vehicle_category","quarter"]).sum().unstack().sort_values(by = "vehicle_category")

In [33]:
quarterly_ev_sales

electric_vehicles_sold                     
quarter                                        Q1     Q2     Q3     Q4
state     vehicle_category                                            
Delhi     2-Wheelers                         9502   6186  11967  10439
Karnataka 2-Wheelers                        34733  29942  35674  47762
Delhi     4-Wheelers                         2273   1651   1868   2838
Karnataka 4-Wheelers                         2903   3359   3131   3485

In [34]:
penetration_rate = raw_data.drop(columns = ["quarter","vehicle_category"]).groupby(["state"]).sum()

In [35]:
penetration_rate["penetration_rates"] = (penetration_rate["electric_vehicles_sold"]/penetration_rate["total_vehicles_sold"])*100

In [36]:
penetration_rate["penetration_rates"].to_frame()

,penetration_rates
state,
Delhi,7.705806
Karnataka,10.176373


In [37]:
penetration_rate_by_category = raw_data.drop(columns = "quarter").groupby(["state","vehicle_category"]).sum()

In [38]:
penetration_rate_by_category["penetration_rates"] = (penetration_rate_by_category["electric_vehicles_sold"]/penetration_rate_by_category["total_vehicles_sold"])*100

In [39]:
penetration_rate_by_category[["penetration_rates"]].sort_values(by = "vehicle_category")

,,penetration_rates
state,vehicle_category,
Delhi,2-Wheelers,9.400866
Karnataka,2-Wheelers,11.573279
Delhi,4-Wheelers,4.290757
Karnataka,4-Wheelers,4.261120


In [40]:
penetration_rate_by_quarter = raw_data.drop(columns = "vehicle_category").groupby(["state","quarter"]).sum()

In [41]:
penetration_rate_by_quarter["penetration_rate"] = (penetration_rate_by_quarter["electric_vehicles_sold"]/penetration_rate_by_quarter["total_vehicles_sold"])*100

In [42]:
penetration_rate_by_quarter["penetration_rate"].unstack()

quarter,Q1,Q2,Q3,Q4
state,,,,
Delhi,8.636244,5.489248,8.071997,8.519690
Karnataka,10.455548,8.783415,9.419009,11.892877


# 6

In [67]:
def cagr(sv,ev,y):
    return round((((ev/sv)**(1/y)) -1)*100)

In [68]:
data = makers.copy()
year = [2022,2024]
top_no = 5

In [71]:
def compounded_annual_growth_rate (dt,years,top_no):
    year_count = year[1]-year[0]
    raw_dt = dt[(dt["vehicle_category"]=="4-Wheelers")&(dt["fiscal_year"].isin(range(year[0],year[1]+1)))][["maker","electric_vehicles_sold","fiscal_year"]]
    top_makers = list(raw_dt.groupby("maker").sum().sort_values(by = "electric_vehicles_sold", ascending = False).head(top_no).index)
    final_dt = raw_dt[raw_dt["maker"].isin(top_makers) & raw_dt["fiscal_year"].isin(year)].groupby(["maker","fiscal_year"]).sum().unstack()
    final_dt["CAGR %"] = cagr(final_dt["electric_vehicles_sold"][2022],final_dt["electric_vehicles_sold"][2024],year_count)
    return final_dt

In [72]:
compounded_annual_growth_rate(data,year,top_no)

electric_vehicles_sold        CAGR %
fiscal_year                           2022   2024       
maker                                                   
BYD India                               33   1466  567.0
Hyundai Motor                          110   1390  255.0
MG Motor                              1647   8829  132.0
Mahindra & Mahindra                   4042  23346  140.0
Tata Motors                          12708  48181   95.0